# Generating Team Data
In this notebook, file we will be generating statistics on a team basis. This will be better for training a model as it will contain avergaes for teams **going into** a match up.

# Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import shutil
import zipfile

from tqdm.notebook import tqdm

pd.set_option("display.max_columns", None)

# Utils

In [2]:
def clean_dir(path):
    """Makes a clean directory, removes all files and folders in the specified path"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Load Data

We first need to load in the data. There is a zip file called `seasons.zip` which we need to unzip and get all the data for. The let's create one giant csv that has all of the games from all seasons.

In [3]:
PATH_TO_ZIP = '../assets/data/seasons.zip'
EXTRACTED_DIR = '../assets/data/'
GAMES_DIR = '../assets/data/seasons/'

def getGames(zip_path=PATH_TO_ZIP,
             extract_dir=EXTRACTED_DIR,
             directory_to_games=GAMES_DIR):
    
    with zipfile.ZipFile(PATH_TO_ZIP, 'r') as zip_ref:
        clean_dir(GAMES_DIR)
        zip_ref.extractall(EXTRACTED_DIR)
    
    dfs = []
    for file in os.listdir(directory_to_games):
        dfs.append(pd.read_csv(os.path.join(directory_to_games, file)))
    
    shutil.rmtree(directory_to_games)
    
    return pd.concat(dfs, ignore_index=True).sort_values(by=['date'], ignore_index=True)

# Generating Moving Averages

Next, we want to create a file for each team that contains all of their stats for each game.

In [4]:
TEAMS = set(getGames()['losing_name']).union(set(getGames()['winning_name']))
TEAMS = [team.strip() for team in sorted(TEAMS)]

In [5]:
original_data = getGames()
original_data["away"] = np.where(original_data["winner"]=="Away",
                                 original_data["winning_name"],
                                 original_data["losing_name"])

original_data["home"] = np.where(original_data["winner"]=="Home",
                                 original_data["winning_name"],
                                 original_data["losing_name"])

# This will help later
reordered_labels = ['date',
                    'location',
                    'losing_abbr',
                    'losing_name',
                    'pace',
                    'winning_abbr',
                    'winning_name',
                    'away',
                    'home',
                    'winner',
                    'away_assist_percentage',
                    'away_assists', 
                    'away_block_percentage', 
                    'away_blocks', 
                    'away_defensive_rating', 
                    'away_defensive_rebound_percentage', 
                    'away_defensive_rebounds',
                    'away_effective_field_goal_percentage',
                    'away_field_goal_attempts',
                    'away_field_goal_percentage',
                    'away_field_goals',
                    'away_free_throw_attempt_rate',
                    'away_free_throw_attempts',
                    'away_free_throw_percentage',
                    'away_free_throws',
                    'away_losses',
                    'away_minutes_played',
                    'away_offensive_rating',
                    'away_offensive_rebound_percentage',
                    'away_offensive_rebounds',
                    'away_personal_fouls',
                    'away_points',
                    'away_steal_percentage',
                    'away_steals',
                    'away_three_point_attempt_rate',
                    'away_three_point_field_goal_attempts',
                    'away_three_point_field_goal_percentage',
                    'away_three_point_field_goals',
                    'away_total_rebound_percentage',
                    'away_total_rebounds',
                    'away_true_shooting_percentage',
                    'away_turnover_percentage',
                    'away_turnovers',
                    'away_two_point_field_goal_attempts',
                    'away_two_point_field_goal_percentage',
                    'away_two_point_field_goals',
                    'away_win_percentage',
                    'away_wins',
                    'home_assist_percentage',
                    'home_assists',
                    'home_block_percentage',
                    'home_blocks',
                    'home_defensive_rating',
                    'home_defensive_rebound_percentage',
                    'home_defensive_rebounds',
                    'home_effective_field_goal_percentage',
                    'home_field_goal_attempts',
                    'home_field_goal_percentage',
                    'home_field_goals',
                    'home_free_throw_attempt_rate',
                    'home_free_throw_attempts',
                    'home_free_throw_percentage',
                    'home_free_throws',
                    'home_losses',
                    'home_minutes_played',
                    'home_offensive_rating',
                    'home_offensive_rebound_percentage',
                    'home_offensive_rebounds',
                    'home_personal_fouls',
                    'home_points',
                    'home_steal_percentage',
                    'home_steals',
                    'home_three_point_attempt_rate',
                    'home_three_point_field_goal_attempts',
                    'home_three_point_field_goal_percentage',
                    'home_three_point_field_goals',
                    'home_total_rebound_percentage',
                    'home_total_rebounds',
                    'home_true_shooting_percentage',
                    'home_turnover_percentage',
                    'home_turnovers',
                    'home_two_point_field_goal_attempts',
                    'home_two_point_field_goal_percentage',
                    'home_two_point_field_goals',
                    'home_win_percentage',
                    'home_wins']

original_data.drop(columns=['away_ranking', 'home_ranking'], inplace=True)
original_data['date'] = pd.to_datetime(original_data['date'])
original_data['date'] = original_data['date'].dt.strftime('%Y-%m-%d')
original_data = original_data[reordered_labels]
original_data.dropna(inplace=True)
original_data.drop_duplicates(inplace=True)

print(f"There are {original_data.shape[0]} examples in the set")

There are 54626 examples in the set


In [6]:
new_labels = ['date',
              'location',
              'away',
              'home',
              'winner',
              'assist_percentage',
              'assists', 
              'block_percentage', 
              'blocks', 
              'defensive_rating', 
              'defensive_rebound_percentage', 
              'defensive_rebounds',
              'effective_field_goal_percentage',
              'field_goal_attempts',
              'field_goal_percentage',
              'field_goals',
              'free_throw_attempt_rate',
              'free_throw_attempts',
              'free_throw_percentage',
              'free_throws',
              'losses',
              'minutes_played',
              'offensive_rating',
              'offensive_rebound_percentage',
              'offensive_rebounds',
              'personal_fouls',
              'points',
              'steal_percentage',
              'steals',
              'three_point_attempt_rate',
              'three_point_field_goal_attempts',
              'three_point_field_goal_percentage',
              'three_point_field_goals',
              'total_rebound_percentage',
              'total_rebounds',
              'true_shooting_percentage',
              'turnover_percentage',
              'turnovers',
              'two_point_field_goal_attempts',
              'two_point_field_goal_percentage',
              'two_point_field_goals',
              'win_percentage',
              'wins']

def generateTeamSats(dataframe, teams=TEAMS, folder='../assets/data/team_data/'):
    """For each team, generate the games that they have played"""
    
    # Clean the folder
    clean_dir(folder)
    
    # Loop through all of the teams
    for team in tqdm(teams, unit="teams"):
        file_name = f'{team}_data.csv'
        path = os.path.join(folder, file_name)
        
        # Get home games and away games
        away_games = dataframe.loc[(dataframe["away"] == team)].copy()
        home_games = dataframe.loc[(dataframe["home"] == team)].copy()
        
        home_games.drop(home_games.filter(regex="away_").columns, axis=1, inplace=True)
        home_games.drop(columns=['losing_abbr',
                                 'losing_name',
                                 'pace',
                                 'winning_abbr',
                                 'winning_name'],
                        inplace=True)
        home_games.columns = new_labels
        
        away_games.drop(away_games.filter(regex="home_").columns, axis=1, inplace=True)
        away_games.drop(columns=['losing_abbr',
                                 'losing_name',
                                 'pace',
                                 'winning_abbr',
                                 'winning_name'],
                        inplace=True)
        away_games.columns = new_labels
        
        # Shaoe for both must match same columns
        assert away_games.shape[1] == home_games.shape[1]
        
        # Join the home games and away games, sort by date
        team_stats = pd.concat([home_games, away_games])
        team_stats.drop_duplicates(inplace=True)
        team_stats.dropna(inplace=True)
        team_stats.sort_values(by=["date"], inplace=True)
        
        # Save the stats
        team_stats.to_csv(path, index=None)

In [7]:
generateTeamSats(original_data)

  0%|          | 0/1146 [00:00<?, ?teams/s]

In [8]:
pd.read_csv('../assets/data/team_data/Yale_data.csv').head()

,date,location,away,home,winner,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,losses,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins
0,2009-11-13,"Arena at Harbor Yard, Bridgeport, Connecticut",Sacred Heart,Yale,Away,53.8,14,0.0,0,115.0,62.1,22,0.433,67,0.388,26,0.463,31,0.903,28,1,200.0,107.5,41.0,12,20,86,15.0,12,0.269,18.0,0.333,6.0,50.0,34,0.526,14.8,14,49.0,0.408,20.0,0.000,0
1,2009-11-16,"Harry A. Gampel Pavilion, Storrs, Connecticut",Hofstra,Yale,Away,70.0,14,4.5,2,94.4,46.2,12,0.434,53,0.377,20,0.434,23,0.739,17,2,200.0,87.5,23.3,7,17,63,18.1,13,0.283,15.0,0.400,6.0,33.9,19,0.493,20.2,14,38.0,0.368,14.0,0.000,0
2,2009-11-17,"Harry A. Gampel Pavilion, Storrs, Connecticut",Yale,Colgate,Away,47.6,10,11.1,2,90.2,68.0,19,0.411,56,0.375,21,0.429,24,0.792,19,2,200.0,106.6,54.5,16,15,65,18.0,11,0.339,19.0,0.211,4.0,60.3,35,0.482,15.3,12,37.0,0.459,17.0,0.333,1
3,2009-11-21,"John J. Lee Ampitheater, New Haven, Connecticut",Quinnipiac,Yale,Away,56.0,14,7.5,4,98.6,42.9,18,0.427,62,0.403,25,0.258,16,0.688,11,3,200.0,88.9,29.7,11,21,64,13.9,10,0.210,13.0,0.231,3.0,36.7,29,0.460,16.9,14,49.0,0.449,22.0,0.250,1
4,2009-11-23,"Mack Sports Complex, Hempstead, New York",Charlotte,Yale,Away,58.3,14,11.1,4,133.3,53.6,16,0.500,54,0.444,24,0.426,23,0.870,20,4,200.0,112.1,37.9,10,21,74,9.1,6,0.278,15.0,0.400,6.0,45.6,26,0.570,15.8,12,39.0,0.462,18.0,0.200,1


In [9]:
def compute_ma(span, teams=TEAMS, team_data_folder='../assets/data/team_data/', dest_folder='../assets/data/team_ma/'):
    """Computes various moving averages for the stats"""
    
    # Clean directory we will be saving the CSVs to
    clean_dir(dest_folder)
    
    for team in tqdm(teams, unit='teams'):
        averages = pd.DataFrame()

        # Load the stats for a given team
        team_stats = pd.read_csv(os.path.join(team_data_folder, f"{team}_data.csv"))

        # Compute the moving averages for the appropriate columns
        for col in team_stats.columns:
            if col in {'date', 'location', 'away', 'home', 'winner'}:
                continue
                
            # Simple moving average
            team_stats[f"{col}_SMA"] = team_stats.loc[:, col].rolling(window=span).mean()
            team_stats[f"{col}_SMA"] = team_stats[f"{col}_SMA"].shift(1)

            # Cumulative moving average
            team_stats[f"{col}_CMA"] = team_stats.loc[:, col].expanding(min_periods=span).mean()
            team_stats[f"{col}_CMA"] = team_stats[f"{col}_CMA"].shift(1)

            # Exponential moving average
            team_stats[f"{col}_EMA"] = team_stats.loc[:, col].ewm(span=span, adjust=False).mean()
            team_stats[f"{col}_EMA"] = team_stats[f"{col}_EMA"].shift(1)

        # Drop any rows with NULL values and save the CSV
        team_stats.dropna(inplace=True)
        team_stats.drop_duplicates(inplace=True)
        team_stats.to_csv(os.path.join(dest_folder, f"{team}_ma.csv"), index=None)

In [10]:
compute_ma(5)

  0%|          | 0/1146 [00:00<?, ?teams/s]

In [11]:
pd.read_csv('../assets/data/team_ma/Yale_ma.csv').head()

,date,location,away,home,winner,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,losses,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins,assist_percentage_SMA,assist_percentage_CMA,assist_percentage_EMA,assists_SMA,assists_CMA,assists_EMA,block_percentage_SMA,block_percentage_CMA,block_percentage_EMA,blocks_SMA,blocks_CMA,blocks_EMA,defensive_rating_SMA,defensive_rating_CMA,defensive_rating_EMA,defensive_rebound_percentage_SMA,defensive_rebound_percentage_CMA,defensive_rebound_percentage_EMA,defensive_rebounds_SMA,defensive_rebounds_CMA,defensive_rebounds_EMA,effective_field_goal_percentage_SMA,effective_field_goal_percentage_CMA,effective_field_goal_percentage_EMA,field_goal_attempts_SMA,field_goal_attempts_CMA,field_goal_attempts_EMA,field_goal_percentage_SMA,field_goal_percentage_CMA,field_goal_percentage_EMA,field_goals_SMA,field_goals_CMA,field_goals_EMA,free_throw_attempt_rate_SMA,free_throw_attempt_rate_CMA,free_throw_attempt_rate_EMA,free_throw_attempts_SMA,free_throw_attempts_CMA,free_throw_attempts_EMA,free_throw_percentage_SMA,free_throw_percentage_CMA,free_throw_percentage_EMA,free_throws_SMA,free_throws_CMA,free_throws_EMA,losses_SMA,losses_CMA,losses_EMA,minutes_played_SMA,minutes_played_CMA,minutes_played_EMA,offensive_rating_SMA,offensive_rating_CMA,offensive_rating_EMA,offensive_rebound_percentage_SMA,offensive_rebound_percentage_CMA,offensive_rebound_percentage_EMA,offensive_rebounds_SMA,offensive_rebounds_CMA,offensive_rebounds_EMA,personal_fouls_SMA,personal_fouls_CMA,personal_fouls_EMA,points_SMA,points_CMA,points_EMA,steal_percentage_SMA,steal_percentage_CMA,steal_percentage_EMA,steals_SMA,steals_CMA,steals_EMA,three_point_attempt_rate_SMA,three_point_attempt_rate_CMA,three_point_attempt_rate_EMA,three_point_field_goal_attempts_SMA,three_point_field_goal_attempts_CMA,three_point_field_goal_attempts_EMA,three_point_field_goal_percentage_SMA,three_point_field_goal_percentage_CMA,three_point_field_goal_percentage_EMA,three_point_field_goals_SMA,three_point_field_goals_CMA,three_point_field_goals_EMA,total_rebound_percentage_SMA,total_rebound_percentage_CMA,total_rebound_percentage_EMA,total_rebounds_SMA,total_rebounds_CMA,total_rebounds_EMA,true_shooting_percentage_SMA,true_shooting_percentage_CMA,true_shooting_percentage_EMA,turnover_percentage_SMA,turnover_percentage_CMA,turnover_percentage_EMA,turnovers_SMA,turnovers_CMA,turnovers_EMA,two_point_field_goal_attempts_SMA,two_point_field_goal_attempts_CMA,two_point_field_goal_attempts_EMA,two_point_field_goal_percentage_SMA,two_point_field_goal_percentage_CMA,two_point_field_goal_percentage_EMA,two_point_field_goals_SMA,two_point_field_goals_CMA,two_point_field_goals_EMA,win_percentage_SMA,win_percentage_CMA,win_percentage_EMA,wins_SMA,wins_CMA,wins_EMA
0,2009-11-24,"Mack Sports Complex, Hempstead, New York",Yale,Elon,Away,48.0,12,10.7,3,98.5,64.7,13,0.520,50,0.500,25,0.380,19,0.895,17,4,200.0,104.5,38.5,8,16,69,18.2,12,0.140,7.0,0.286,2.0,48.8,21,0.584,20.4,15,43.0,0.535,23.0,0.333,2,57.14,57.140000,56.470370,13.2,13.200000,13.407407,6.84,6.840000,7.455556,2.4,2.400000,2.716049,106.30,106.300000,111.746914,54.56,54.560000,54.303704,17.4,17.400000,17.679012,0.4410,0.441000,0.450840,58.4,58.400,58.543210,0.3974,0.397400,0.406988,23.2,23.200000,23.777778,0.4020,0.402000,0.397210,23.4,23.400000,23.172840,0.7984,0.798400,0.811580,19.0,19.000000,19.135802

# Getting the Data Ready for Testing
Now that we have computed the exponential moving average for each team, we need to joing them back into one table so that it is easier to train models on.

In [12]:
games = original_data.copy()
num_rows = games.shape[0]

# We can uniquely identify a game given the teams, the date, and the game number.
games = games.filter(["date", "location", "home", "away", "winner"])
games.drop_duplicates()

# Just to make sure that there is no lost data
assert games.shape[0] == num_rows

# Arrays to hold the home and away dataframes
homes=[]
aways=[]

# Add the moving averages
for team in tqdm(TEAMS, unit='teams'):
    # load in the  file
    team_avgs = pd.read_csv(f'../assets/data/team_ma/{team}_ma.csv')
    team_avgs.drop(new_labels[5:], axis=1, inplace=True)
    
    # Merge in visiting team stats
    away = pd.merge(games,
                    team_avgs.loc[team_avgs["away"] == team],
                    on=["date", "location", "home", "away", 'winner'],
                    how="left")
    
    # Drop any rows with NULL values and drop redudant columns
    away.dropna(inplace=True)
    away.drop_duplicates(inplace=True)
    
    # Rename the columns and append to the array
    new_away_cols = list(away.columns)[:5] + ["away_" + col for col in list(away.columns)[5:]]
    away.columns = new_away_cols
    aways.append(away)
    
    # Merge in home team stats
    home = pd.merge(games,
                    team_avgs.loc[team_avgs["home"] == team],
                    on=["date", "location", "home", "away", 'winner'],
                    how="left")
    
    # Drop any rows with NULL values and drop redudant columns
    home.dropna(inplace=True)
    home.drop_duplicates(inplace=True)
    
    # Rename the columns and append to the array
    new_home_cols = list(home.columns)[:5] + ["home_" + col for col in list(home.columns)[5:]]
    home.columns = new_home_cols
    homes.append(home)
    
    # Sanity check, make sure that the column sizes match for both home and away dataframes
    assert home.shape[1] == away.shape[1]

  0%|          | 0/1146 [00:00<?, ?teams/s]

In [13]:
# Join the tables on the common attributes and sort by the Date and the Number of game
data = pd.merge(pd.concat(homes),
                pd.concat(aways),
                on=["date", "location", "home", "away", "winner"]).sort_values(["date"]).reset_index(drop=True)

data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.head(10)

,date,location,home,away,winner,home_assist_percentage_SMA,home_assist_percentage_CMA,home_assist_percentage_EMA,home_assists_SMA,home_assists_CMA,home_assists_EMA,home_block_percentage_SMA,home_block_percentage_CMA,home_block_percentage_EMA,home_blocks_SMA,home_blocks_CMA,home_blocks_EMA,home_defensive_rating_SMA,home_defensive_rating_CMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_SMA,home_defensive_rebound_percentage_CMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_SMA,home_defensive_rebounds_CMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_SMA,home_effective_field_goal_percentage_CMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_SMA,home_field_goal_attempts_CMA,home_field_goal_attempts_EMA,home_field_goal_percentage_SMA,home_field_goal_percentage_CMA,home_field_goal_percentage_EMA,home_field_goals_SMA,home_field_goals_CMA,home_field_goals_EMA,home_free_throw_attempt_rate_SMA,home_free_throw_attempt_rate_CMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_SMA,home_free_throw_attempts_CMA,home_free_throw_attempts_EMA,home_free_throw_percentage_SMA,home_free_throw_percentage_CMA,home_free_throw_percentage_EMA,home_free_throws_SMA,home_free_throws_CMA,home_free_throws_EMA,home_losses_SMA,home_losses_CMA,home_losses_EMA,home_minutes_played_SMA,home_minutes_played_CMA,home_minutes_played_EMA,home_offensive_rating_SMA,home_offensive_rating_CMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_SMA,home_offensive_rebound_percentage_CMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_SMA,home_offensive_rebounds_CMA,home_offensive_rebounds_EMA,home_personal_fouls_SMA,home_personal_fouls_CMA,home_personal_fouls_EMA,home_points_SMA,home_points_CMA,home_points_EMA,home_steal_percentage_SMA,home_steal_percentage_CMA,home_steal_percentage_EMA,home_steals_SMA,home_steals_CMA,home_steals_EMA,home_three_point_attempt_rate_SMA,home_three_point_attempt_rate_CMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_SMA,home_three_point_field_goal_attempts_CMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_SMA,home_three_point_field_goal_percentage_CMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_SMA,home_three_point_field_goals_CMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_SMA,home_total_rebound_percentage_CMA,home_total_rebound_percentage_EMA,home_total_rebounds_SMA,home_total_rebounds_CMA,home_total_rebounds_EMA,home_true_shooting_percentage_SMA,home_true_shooting_percentage_CMA,home_true_shooting_percentage_EMA,home_turnover_percentage_SMA,home_turnover_percentage_CMA,home_turnover_percentage_EMA,home_turnovers_SMA,home_turnovers_CMA,home_turnovers_EMA,home_two_point_field_goal_attempts_SMA,home_two_point_field_goal_attempts_CMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_SMA,home_two_point_field_goal_percentage_CMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_SMA,home_two_point_field_goals_CMA,home_two_point_field_goals_EMA,home_win_percentage_SMA,home_win_percentage_CMA,home_win_percentage_EMA,home_wins_SMA,home_wins_CMA,home_wins_EMA,away_assist_percentage_SMA,away_assist_percentage_CMA,away_assist_percentage_EMA,away_assists_SMA,away_assists_CMA,away_assists_EMA,away_block_percentage_SMA,away_block_percentage_CMA,away_block_percentage_EMA,away_blocks_SMA,away_blocks_CMA,away_blocks_EMA,away_defensive_rating_SMA,away_defensive_rating_CMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_SMA,away_defensive_rebound_percentage_CMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_SMA,away_defensive_rebounds_CMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_SMA,away_effective_field_goal_percentage_CMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_SMA,away_field_goal_attempts_CMA,away_field_goal_attempts_EMA,away_field_goal_perc

In [14]:
# Save the dataframe that we just generated

clean_dir('../assets/data/cleaned_data')
data.to_csv('../assets/data/cleaned_data/cleaned_data.csv', index=None)